In [1]:
from fair_util import BMInterface, BMMetrics
from fair_bm import BMManager
from fair_grid import dummy_model
import pandas as pd
import numpy as np 

C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\enemy\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_disc


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
wids_train_df = pd.read_csv('./data/wids_pre/wids_train.csv')
wids_val_df = pd.read_csv('./data/wids_pre/wids_val.csv')
wids_test_df = pd.read_csv('./data/wids_pre/wids_test.csv')

In [3]:
# Defining the name of the label column
label_name = 'DiagPeriodL90D'
# Defininf the name of the privileged group
sensitive_attribute = ['patient_age']

bmI = BMInterface(wids_train_df, wids_val_df, wids_test_df, label_name, sensitive_attribute)

In [4]:
x_train, y_train = bmI.get_train_xy() 
x_val , y_val = bmI.get_val_xy()
x_test, y_test = bmI.get_test_xy()

### Model hyperparameter tunning

In [38]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(eval_metric='Accuracy',
                         depth =  4,
                         learning_rate = 0.01,
                         iterations = 10)

cat.fit(x_train, y_train, eval_set=(x_val, y_val))


0:	learn: 0.8103832	test: 0.8133125	best: 0.8133125 (0)	total: 136ms	remaining: 1.23s
1:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 144ms	remaining: 578ms
2:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 152ms	remaining: 356ms
3:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 160ms	remaining: 240ms
4:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 166ms	remaining: 166ms
5:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 172ms	remaining: 114ms
6:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 178ms	remaining: 76.1ms
7:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 182ms	remaining: 45.4ms
8:	learn: 0.8107175	test: 0.8148726	best: 0.8153926 (1)	total: 186ms	remaining: 20.7ms
9:	learn: 0.8106061	test: 0.8153926	best: 0.8153926 (1)	total: 191ms	remaining: 0us

bestTest = 0.8153926157
bestIteration = 1

Shrink model to first 2 iterations.


In [40]:
y_val_pred = cat.predict_proba(x_val)
y_test_pred = cat.predict_proba(x_test)

In [28]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# Training Logistic Regression model
model = LogisticRegression(solver='liblinear', random_state=42)

LR_model = model.fit(x_train, y_train, sample_weight=bmI.get_train_BLD().instance_weights)
y_val_pred = LR_model.predict_proba(x_val)
y_test_pred = LR_model.predict_proba(x_test)

In [42]:
privileged_group = [{'patient_age': 1}]
unprivileged_group = [{'patient_age': 0}]

bmM = BMMetrics(bmI, model.classes_, y_val_pred, y_test_pred, privileged_group, unprivileged_group)

In [43]:
report_dict = bmM.get_report()

print("Balanced Accuracy", report_dict['balanced_acc'])
print("Accuracy", report_dict['acc'])
print("Precision", report_dict['precision'])
print("Recall", report_dict['recall'])
print("F1 Score", report_dict['f1'])
print('-----------------------------')    
print("Equal opportunity difference", report_dict['eq_opp_diff'])
print("Average odds difference", report_dict['avg_odd_diff'])
print("Statistical parity difference", report_dict['spd'])
print("Disparate Impact", report_dict['disparate_impact'])
print("Theil index", report_dict['theil_idx'])

Balanced Accuracy 0.7667538462069989
Accuracy 0.817047817047817
Precision 0.7870182555780934
Recall 0.9691923397169026
F1 Score 0.8686567164179105
-----------------------------
Equal opportunity difference -0.03111858704793946
Average odds difference -0.04789587210533275
Statistical parity difference -0.10718029350104818
Disparate Impact 0.8775082359988021
Theil index 0.0633521775909673


In [46]:
score_dict = bmM.get_score()
score_dict

{'overall_score': 0.354,
 'metric_evaluations': {'EOD': True,
  'AOD': True,
  'SPD': False,
  'DI': True,
  'TI': True},
 'deviations': {'EOD': 0.311,
  'AOD': 0.479,
  'SPD': 1.072,
  'DI': 0.612,
  'TI': 0.253},
 'is_fair': False}

In [ ]:
bmMR = BMManager(bmI, bmM)

bmMR.pos_ROC()

x_train, y_train = bmI.get_train_xy() 

LR_model = model.fit(x_train, y_train, sample_weight=bmI.get_train_BLD().instance_weights)
y_val_pred = LR_model.predict_proba(x_val)
y_test_pred = LR_model.predict_proba(x_test)

In [ ]:
pred_val_BLD = bmI.get_val_BLD().copy(deepcopy=True)
pred_val_BLD.labels = y_val_pred

pred_BLD = bmI.get_test_BLD().copy(deepcopy=True)
pred_BLD.label = y_test_pred

In [ ]:
bmM.set_new_pred(y_val_pred, y_test_pred)
bmM.get_report()

In [ ]:
score_dict = bmM.get_score()
score_dict

In [ ]:
bmI.restore_BLD()